### **Big-M Method**

In [3]:
import copy

import numpy as np
from tabulate import tabulate

In [5]:
def visualize_tabulation(tabulation, all_vars, basic_vars, big_M=False):

    tabulation = np.round(tabulation, 4)

    header = np.concatenate(([], all_vars, ["sol"]))

    if big_M:
        # if big-M method is implemented, combine the first two rows to create the objective function row
        obj_func_row = []
        for i in range(tabulation.shape[1]):
            entry = ''
            if tabulation[0][i] != 0:
                entry += str(tabulation[0][i])
            if tabulation[1][i] != 0:
                if tabulation[1][i] > 0 and entry != '': entry += '+'
                entry += str(tabulation[1][i]) + 'M'
            if entry == '': entry = 0
            obj_func_row.append(entry)
        obj_func_row = np.reshape(obj_func_row, (1, -1))
        tabulation = np.concatenate((obj_func_row, tabulation[2: ]), axis=0)
        basic_vars = np.concatenate(([basic_vars[0]], basic_vars[2: ]))

    print(tabulate(np.concatenate((np.array(basic_vars).reshape(-1, 1), tabulation), axis=1), headers=header, stralign='center', tablefmt='simple_grid'))

In [1]:
# no. of decision variables (N)
NO_OF_DECISION_VARS = 4

# no. of conditions 
NO_OF_CONDITIONS = 2

# --------------------------------- objective function ---------------------------------
# specify the objective function in the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>]
OBJ_FUNC = [24, 6, 1, 2]

# -------------------------------- type of optimization --------------------------------
OPT_TYPE = "MIN" # or "MIN" 

# ------------------------------------- conditions -------------------------------------
# specify each condition in a new row with the format [<coefficient_1>, <coefficient_2>, ..., <coefficient_N>, 'operation', <solution>]
# the operation can be one of the '>=', '<=', or '='
conditions = [
    [  6, 1, -1, 0, '>=', 5],
    [  4, 5,  1, 1, '>=', 4],
]

# define M value
M = 1e6

- To implement big-M method, we use two rows for the objective function; one for the constant term and the other for `M` terms. 

In [6]:
# create the variables and generate the initial tabulation
OBJ_FUNC = np.array(OBJ_FUNC)
conditions = np.array(conditions)

# extract the operations
operations = list(conditions[:, -2])
no_of_less_than_or_equals    = operations.count('<=')
no_of_greater_than_or_equals = operations.count('>=')
no_of_equalities             = operations.count('=')

# find the size of a row
row_size = 1 + NO_OF_DECISION_VARS + no_of_less_than_or_equals + 2*no_of_greater_than_or_equals + no_of_equalities + 1

# find the column size
col_size = 2 + NO_OF_CONDITIONS # no. of basic vars = 2 (for objective function) + no. of conditions

# initialize the initial tabulation
initial_tabulation = np.zeros((col_size, row_size))

var_symbol_arr = []
basic_var_symbol_arr = []

OBJ_FUNC_VAR_SYMBOL = "P"
DECISION_VAR_SYMBOL = "X"
SLACK_VAR_SYMBOL    = "S"
SURPLUS_VAR_SYMBOL  = "S"
ARTIFICIAL_VAR_SYMBOL = "A"

var_symbol_arr.append(OBJ_FUNC_VAR_SYMBOL)

# create the objective function row
basic_var_symbol_arr.extend([OBJ_FUNC_VAR_SYMBOL] * 2)
initial_tabulation[0][0] = 1
initial_tabulation[0][1 : len(OBJ_FUNC)+1] = -OBJ_FUNC

for i in range(NO_OF_DECISION_VARS):
    var_symbol_arr.append(DECISION_VAR_SYMBOL + str(i+1))

# create all the other basic variable rows
index = 1 + NO_OF_DECISION_VARS
artificial_row_idxs = []
for i in range(NO_OF_CONDITIONS):

    initial_tabulation[i+2][1 : NO_OF_DECISION_VARS+1] = conditions[i][ :-2]
    initial_tabulation[i+2][-1] = conditions[i][-1]
    operation = operations[i]

    if operation == '<=':
        # introduce a slack variable (a basic variable)
        var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] = 1
        index += 1
        basic_var_symbol_arr.append(SLACK_VAR_SYMBOL + str(i+1))
    elif operation == ">=":
        # introduce a surplus  (a non-basic variable) and an artificial variable (a basic variable) 
        var_symbol_arr.append(SURPLUS_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] = -1
        index += 1
        var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] =  1
        initial_tabulation[1][index] = -1 if OPT_TYPE == 'MIN' else 1 # carefull!!!
        index += 1
        basic_var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        artificial_row_idxs.append(i+2)
    elif operation == "=":
        # introduce an artificial variable (a basic variable)
        var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        initial_tabulation[i+2][index] = 1
        initial_tabulation[1][index] = -1 if OPT_TYPE == 'MIN' else 1 # careful!!!
        index += 1
        basic_var_symbol_arr.append(ARTIFICIAL_VAR_SYMBOL + str(i+1))
        artificial_row_idxs.append(i+2)

print("variables:", var_symbol_arr)
print("basic variables:", basic_var_symbol_arr)

visualize_tabulation(initial_tabulation, var_symbol_arr, basic_var_symbol_arr, big_M=True)

variables: ['P', 'X1', 'X2', 'X3', 'X4', 'S1', 'A1', 'S2', 'A2']
basic variables: ['P', 'P', 'A1', 'A2']
┌────┬─────┬──────┬──────┬──────┬──────┬──────┬───────┬──────┬───────┬───────┐
│    │   P │   X1 │   X2 │   X3 │   X4 │   S1 │  A1   │   S2 │  A2   │   sol │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┼──────┼───────┼───────┤
│ P  │   1 │  -24 │   -6 │   -1 │   -2 │    0 │ -1.0M │    0 │ -1.0M │     0 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┼──────┼───────┼───────┤
│ A1 │   0 │    6 │    1 │   -1 │    0 │   -1 │  1.0  │    0 │  0.0  │     5 │
├────┼─────┼──────┼──────┼──────┼──────┼──────┼───────┼──────┼───────┼───────┤
│ A2 │   0 │    4 │    5 │    1 │    1 │    0 │  0.0  │   -1 │  1.0  │     4 │
└────┴─────┴──────┴──────┴──────┴──────┴──────┴───────┴──────┴───────┴───────┘


In [7]:
# make the artificial variable columns pivot vectors
for artificial_row_idx in artificial_row_idxs:
    if OPT_TYPE == "MIN": initial_tabulation[1] += initial_tabulation[artificial_row_idx]
    else:                 initial_tabulation[1] -= initial_tabulation[artificial_row_idx]

print("="*20 + " Initial Feasible Solution " + "="*20)
visualize_tabulation(initial_tabulation, var_symbol_arr, basic_var_symbol_arr, big_M=True)

==================== Initial Feasible Solution ====================
┌────┬─────┬─────────────┬───────────┬──────┬───────────┬───────┬──────┬───────┬──────┬───────┐
│    │   P │     X1      │    X2     │   X3 │    X4     │  S1   │   A1 │  S2   │   A2 │  sol  │
├────┼─────┼─────────────┼───────────┼──────┼───────────┼───────┼──────┼───────┼──────┼───────┤
│ P  │   1 │ -24.0+10.0M │ -6.0+6.0M │   -1 │ -2.0+1.0M │ -1.0M │    0 │ -1.0M │    0 │ 9.0M  │
├────┼─────┼─────────────┼───────────┼──────┼───────────┼───────┼──────┼───────┼──────┼───────┤
│ A1 │   0 │     6.0     │    1.0    │   -1 │    0.0    │ -1.0  │    1 │  0.0  │    0 │  5.0  │
├────┼─────┼─────────────┼───────────┼──────┼───────────┼───────┼──────┼───────┼──────┼───────┤
│ A2 │   0 │     4.0     │    5.0    │    1 │    1.0    │  0.0  │    0 │ -1.0  │    1 │  4.0  │
└────┴─────┴─────────────┴───────────┴──────┴───────────┴───────┴──────┴───────┴──────┴───────┘


In [8]:
iteration_no = 0
tabulation = copy.deepcopy(initial_tabulation)

for _ in range(10): # *****

    # ====================================== ENTERING VARIABLE ======================================
    obj_func_row = tabulation[0] + tabulation[1] * M

    if OPT_TYPE == "MAX":
        if not (obj_func_row[1:-1] < 0).any():
            # raise Exception(f"there are no negative values in the objective row...")
            print(f"***** there are no more negative values in the objective row; tabulation is optimal... *****\n")
            break
        pivot_col_value = np.min(obj_func_row[1:-1]) # highest negative value
    else: # OPT_TYPE == "MIN"
        if not (obj_func_row[1:-1] > 0).any():
            # raise Exception(f"there are no positive values in the objective row...")
            print(f"***** there are no more positive values in the objective row; tabulation is optimal... *****\n")
            break
        pivot_col_value = np.max(obj_func_row[1:-1]) # highest positive value

    iteration_no += 1
    print("="*20 + f" iteration no.: {iteration_no} " + "="*20)

    pivot_col_idx = list(obj_func_row[1:-1]).index(pivot_col_value) + 1
    # -----------------------------------------------------------------------------------------------

    entering_var = var_symbol_arr[pivot_col_idx]
    print(f"entering var: {entering_var}")
    pivot_col = tabulation[:, pivot_col_idx]
    print(f"pivot col: {pivot_col}")

    # find the ratio column
    solution  = tabulation[:, -1]
    print(f"solution : {solution}")
    ratio_col = solution / pivot_col
    print(f"ratio col: {ratio_col}")

    # ====================================== LEAVING VARIABLE =======================================
    # find the lowest positive value in the ratio column
    if not (ratio_col[2: ] > 0).any():
        raise Exception(f"there is no positive value in the ratio column; therefore, NO FEASIBLE SOLUTION EXISTS...")

    pivot_row_value = np.min(ratio_col[2: ][ratio_col[2: ] > 0])
    pivot_row_idx = list(ratio_col[2: ]).index(pivot_row_value) + 2
    # -----------------------------------------------------------------------------------------------

    leaving_var = basic_var_symbol_arr[pivot_row_idx]
    print(f"leaving var : {leaving_var}")
    pivot_row = tabulation[pivot_row_idx]
    print(f"pivot row: {pivot_row}")

    # replace the leaving variable with the entering variable
    basic_var_symbol_arr[pivot_row_idx] = entering_var
    print(f"new basic variables: {basic_var_symbol_arr}")

    # convert the pivot column to a pivot vector

    # verify that the pivot column is convertible to a pivot vector
    if pivot_row[pivot_col_idx] == 0:
        raise Exception(f"the pivot column (var '{entering_var}') at index {pivot_col_idx} is not convertible to a pivot vector...")

    # transform the pivot row
    pivot_row_ = pivot_row / pivot_row[pivot_col_idx]
    tabulation[pivot_row_idx] = pivot_row_

    # transform all the other rows
    for i in range(col_size):

        if i == pivot_row_idx:
            continue

        row = tabulation[i]
        tabulation[i] = row - row[pivot_col_idx] * pivot_row_

    visualize_tabulation(tabulation, all_vars=var_symbol_arr, basic_vars=basic_var_symbol_arr, big_M=True)

    # the values in the solution column except the one in the objective function row must be non-negative
    is_feasible = (tabulation[:, -1][2: ] >= 0).all()

    if not is_feasible:
        raise Exception(f"the tabulation is not feasible...")
    print(f"the tabulation is feasible.")

    print('\n')

# check for many solutions
non_basic_vars = list(set(var_symbol_arr) - set(basic_var_symbol_arr))
non_basic_vars.sort()
for non_basic_var in non_basic_vars:
    # find the objective row value for each non-basic variable
    print(non_basic_var, obj_func_row[var_symbol_arr.index(non_basic_var)])
    if obj_func_row[var_symbol_arr.index(non_basic_var)] == 0:
        raise Exception(f"non-basic variable '{non_basic_var}' has a zero value in the objective row; therefore, MANY SOLUTIONS EXISTS...")

==================== iteration no.: 1 ====================
entering var: X1
pivot col: [-24.  10.   6.   4.]
solution : [0. 9. 5. 4.]
ratio col: [-0.          0.9         0.83333333  1.        ]
leaving var : A1
pivot row: [ 0.  6.  1. -1.  0. -1.  1.  0.  0.  5.]
new basic variables: ['P', 'P', 'X1', 'A2']
┌────┬─────┬──────┬──────────────┬──────────────┬───────────┬──────────────┬─────────────┬───────┬──────┬──────────────┐
│    │   P │   X1 │      X2      │      X3      │    X4     │      S1      │     A1      │  S2   │   A2 │     sol      │
├────┼─────┼──────┼──────────────┼──────────────┼───────────┼──────────────┼─────────────┼───────┼──────┼──────────────┤
│ P  │   1 │    0 │ -2.0+4.3333M │ -5.0+1.6667M │ -2.0+1.0M │ -4.0+0.6667M │ 4.0-1.6667M │ -1.0M │    0 │ 20.0+0.6667M │
├────┼─────┼──────┼──────────────┼──────────────┼───────────┼──────────────┼─────────────┼───────┼──────┼──────────────┤
│ X1 │   0 │    1 │    0.1667    │   -0.1667    │    0.0    │   -0.1667    │   0.1667 